In [1]:
import os
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from torch import nn, optim
from PIL import Image
from tqdm import tqdm
from sklearn.model_selection import train_test_split

# Define Custom Dataset
class CustomDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None, mode='train'):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.mode = mode
        
        if self.mode == 'train':
            self.classes = sorted(self.data['city'].unique())
            self.class_to_idx = {cls_name: idx for idx, cls_name in enumerate(self.classes)}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.data.iloc[idx]['filename'])
        image = Image.open(img_name).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        if self.mode == 'train':
            city = self.data.iloc[idx]['city']
            label = self.class_to_idx[city]
            return image, label
        else:
            return image, self.data.iloc[idx]['filename']


In [2]:
device = "mps" if torch.backends.mps.is_available() else ("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [3]:
train_csv = '/kaggle/input/datathon-ai-qualification-round/train_data.csv'
test_csv = '/kaggle/input/datathon-ai-qualification-round/test.csv'
train_dir = '/kaggle/input/datathon-ai-qualification-round/train/train'  # Adjust this to the correct path
test_dir = '/kaggle/input/datathon-ai-qualification-round/test/test'


# Split training CSV into train and validation sets
df = pd.read_csv(train_csv)
train_df, val_df = train_test_split(df, test_size=0.15, stratify=df['city'], random_state=42)

# Save temporary CSVs for train and val to load with dataset class
train_split_csv = "train_split.csv"
val_split_csv = "val_split.csv"
train_df.to_csv(train_split_csv, index=False)
val_df.to_csv(val_split_csv, index=False)

In [4]:

# Transform definitions
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

train_transforms = transforms.Compose([
    transforms.Resize((380, 380)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

val_transforms = transforms.Compose([
    transforms.Resize((380, 380)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

test_transforms = transforms.Compose([
    transforms.Resize((380, 380)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

In [5]:
train_dataset = CustomDataset(csv_file=train_split_csv, root_dir=train_dir, transform=train_transforms, mode='train')
val_dataset = CustomDataset(csv_file=val_split_csv, root_dir=train_dir, transform=val_transforms, mode='train')
test_dataset = CustomDataset(csv_file=test_csv, root_dir=test_dir, transform=test_transforms, mode='test')

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)


In [6]:
import os
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from torch import nn, optim
from PIL import Image
from tqdm import tqdm
from torch.optim.lr_scheduler import ReduceLROnPlateau

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_classes = len(train_dataset.class_to_idx)

# Load the EfficientNet-B2 model with pretrained weights
model = models.efficientnet_b2(weights=models.EfficientNet_B2_Weights.IMAGENET1K_V1)

# Freeze all layers initially
for param in model.parameters():
    param.requires_grad = False

# Unfreeze the last two blocks and classifier
for param in model.classifier.parameters():
    param.requires_grad = True
for param in model.features[6:].parameters():  # Unfreeze last two blocks (6 and 7)
    param.requires_grad = True

# Modify the classification head
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss()

# Optimizer with different learning rates and weight decay
optimizer = optim.Adam([
    {'params': model.features[6:].parameters(), 'lr': 1e-5, 'weight_decay': 1e-5},  # Lower LR, weight decay for features
    {'params': model.classifier.parameters(), 'lr': 1e-4, 'weight_decay': 1e-5}   # Higher LR, weight decay for classifier
])

# Learning Rate Scheduler (ReduceLROnPlateau)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)


Downloading: "https://download.pytorch.org/models/efficientnet_b2_rwightman-c35c1473.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b2_rwightman-c35c1473.pth
100%|██████████| 35.2M/35.2M [00:00<00:00, 214MB/s]
/opt/conda/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [7]:
epochs = 20 # Increased epochs

best_val_loss = float('inf') # For early stopping
patience_counter = 0

for epoch in range(epochs):
    # Training phase
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for images, labels in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{epochs}"):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
    epoch_loss = running_loss / total
    epoch_acc = correct / total

    # Validation phase
    model.eval()
    val_running_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            val_loss = criterion(outputs, labels)
            val_running_loss += val_loss.item() * images.size(0)
            _, val_preds = torch.max(outputs, 1)
            val_correct += (val_preds == labels).sum().item()
            val_total += labels.size(0)
    val_epoch_loss = val_running_loss / val_total
    val_epoch_acc = val_correct / val_total

    scheduler.step(val_epoch_loss) # Step the LR scheduler

    print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc*100:.2f}% | "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc*100:.2f}%")
    
    # Early Stopping
    if val_epoch_loss < best_val_loss:
        best_val_loss = val_epoch_loss
        torch.save(model.state_dict(), "best_model.pth") # Save the best model
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= 5: # Stop if no improvement for 5 epochs
            print("Early stopping triggered.")
            break

Training Epoch 1/20:  17%|█▋        | 32/186 [00:25<02:02,  1.26it/s]


KeyboardInterrupt: 

In [ ]:
# Load best model
model.load_state_dict(torch.load("best_model.pth"))
# ... (Your inference code)# Inference on Test
model.eval()
predictions = []

idx_to_class = {v: k for k, v in train_dataset.class_to_idx.items()}

with torch.no_grad():
    for images, file_names in tqdm(test_loader, desc="Predicting on Test Set"):
        images = images.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        
        for f, p in zip(file_names, preds):
            city_name = idx_to_class[p.item()]
            predictions.append((f, city_name))

# Save predictions
pred_df = pd.DataFrame(predictions, columns=["filename", "city"])
pred_df.to_csv("test_predictionsefficiencynet32.csv", index=False)
print("Predictions saved to test_predictionsefficiencynet32.csv")

